# Uitwerkingen opgaven data-analyse

## Opgave 1.3

We maken hier gebruik van de error function, gedefinieerd in `scipy.special.erf`. Er is ook een complementaire error function, gedefinieerd als $\mathrm{erfc} = 1 - \mathrm{erf}$. Vergeet niet de factor $\sqrt{2}$, aangezien $\mathrm{erf}$ een vereenvoudigde functie is, en het argument nog omgerekend moet worden.

De waarschijnlijkheid dat een waarde méér dan $1.23\sigma$ van het gemiddelde afligt is gegeven door:

In [1]:
from math import sqrt
from scipy.special import erf, erfc

In [2]:
erfc(1.23 / sqrt(2))

0.21869710485138394

De waarschijnlijkheid dat een waarde méér dan $2.43\sigma$ *boven* het gemiddelde ligt is gegeven door:

In [3]:
erfc(2.43 / sqrt(2)) / 2

0.0075494114163092065

De waarschijnlijkheid dat een waarde meer dan $0.5\sigma$, maar minder dan $1.5\sigma$ van het gemiddelde verwijderd is, wordt gegeven door:

In [4]:
erf(1.5 / sqrt(2)) - erf(.5 / sqrt(2))

0.4834606749142577

We willen weten binnen hoeveel standaarddeviaties van het gemiddelde de waarschijnlijkheid dat een waarde gevonden wordt gelijk is aan $50\,\%$. Gebruik hiervoor de inverse error function `erfinv`. Vergeet de factor $\sqrt{2}$ niet!

In [5]:
from scipy.special import erfinv

In [6]:
erfinv(.5) * sqrt(2)

0.67448975019608171